In [ ]:
from ByBit import ByBitOptionData, recommend_option_position, ByBitAPI


In [ ]:
prod_url = "https://api.bybit.com"
ticker_endpoint = "/v5/market/tickers"
btc_params = {'category': "option",
              'baseCoin': "BTC"}

daily_delta_limit: float = 0.10  # Daily Delta Deviation Limit
weekly_delta_limit: float = 0.10  # Weekly Delta Deviation Limit
monthly_delta_limit: float = 0.10  # Monthly Delta Deviation Limit
quarterly_delta_limit: float = 0.10  # Quarterly Delta Deviation Limit

In [ ]:
ByBit = ByBitOptionData(api_url=prod_url, api_endpoint=ticker_endpoint, api_parameters=btc_params)

ByBitAPI = ByBitAPI(default_quantity=1.0,
                    api_url="https://api-demo.bybit.com",
                    api_key="6TzoOeOuyIDiJN9cOu",
                    api_secret="JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",
                    baseCoin="BTC",
                    settleCoin="USDT")

In [ ]:
await ByBit.fetch_ByBit_ticker_data()
await ByBit.format_the_dataframe()

(current_daily, next_daily, next_to_next_daily,
 current_weekly, next_weekly, next_to_next_weekly,
 current_monthly, next_monthly, next_to_next_monthly,
 current_quarterly, next_quarterly, next_to_next_quarterly, expiry) = await ByBit.segregate_options_expiry()


In [ ]:
json_fut = await ByBitAPI.get_PerpFutures_Position()
json_fut

In [ ]:
len(json_fut.json()["result"]["list"])

In [ ]:
json_data = await ByBitAPI.get_option_positions()
position_df = await ByBitAPI.format_option_position_dataframe()

In [ ]:
async def set_expiry_delta_and_compute_hedging():
    expiry["daily"]["delta_limit"] = float(daily_delta_limit)
    expiry["weekly"]["delta_limit"] = float(weekly_delta_limit)
    expiry["monthly"]["delta_limit"] = float(monthly_delta_limit)
    expiry["quarterly"]["delta_limit"] = float(quarterly_delta_limit)

    position_df.loc[position_df.expiry_type == "daily", "expiry_delta"] = daily_delta_limit
    position_df.loc[position_df.expiry_type == "weekly", "expiry_delta"] = weekly_delta_limit
    position_df.loc[position_df.expiry_type == "monthly", "expiry_delta"] = monthly_delta_limit
    position_df.loc[position_df.expiry_type == "quarterly", "expiry_delta"] = quarterly_delta_limit

    for i in range(len(position_df)):
        print(f"Delta Hedging : {abs(round(position_df.loc[i, "expiry_delta"], 6))}",
              f"Expiry Delta {abs(round(position_df.loc[i, "expiry_delta"], 6))}")
        if abs(round(position_df.loc[i, "delta_hedging"], 6)) > abs(round(position_df.loc[i, "expiry_delta"], 6)):
            print(f"Hedging Required !!! ")
            position_df.loc[i, "hedging_required"] = True
            position_df.loc[i, "PerpFutureQty"] = position_df.loc[i, "delta_hedging"]
        else:
            print(f"Hedging Not Required !!! ")
            position_df.loc[i, "hedging_required"] = False
            position_df.loc[i, "PerpFutureQty"] = position_df.loc[i, "delta_hedging"]



In [ ]:
await set_expiry_delta_and_compute_hedging()

In [ ]:
async def check_the_position_availablity():
    ## Update the Daily Expiry Dictionary
    # Update the Daily : Current Daily
    if len(position_df.loc[position_df.expiry == expiry["daily"]["current"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["daily"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["daily"]["current"]["date"], "expiry_type"] = "daily"
        expiry["daily"]["current"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["daily"]["current"]["date"]}")

    # Update the Monthly : Next Daily
    if len(position_df.loc[position_df.expiry == expiry["daily"]["next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["daily"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["daily"]["next"]["date"], "expiry_type"] = "daily"
        expiry["daily"]["next"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["daily"]["next"]["date"]}")

    # Update the Monthly : Next_to_Next Daily
    if len(position_df.loc[position_df.expiry == expiry["daily"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["daily"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["daily"]["next_to_next"]["date"], "expiry_type"] = "daily"
        expiry["daily"]["next_to_next"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["daily"]["next_to_next"]["date"]}")

    ## Update the Weekly Expiry Dictionary 
    # Update the Weekly : Current Weekly
    if len(position_df.loc[position_df.expiry == expiry["weekly"]["current"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["weekly"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["weekly"]["current"]["date"], "expiry_type"] = "weekly"
        expiry["weekly"]["current"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["weekly"]["current"]["date"]}")

    # Update the Weekly : Next Weekly
    if len(position_df.loc[position_df.expiry == expiry["weekly"]["next"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["weekly"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["weekly"]["next"]["date"], "expiry_type"] = "weekly"
        expiry["weekly"]["next"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["weekly"]["next"]["date"]}")

    # Update the Weekly : Next_to_Next Weekly
    if len(position_df.loc[position_df.expiry == expiry["weekly"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["weekly"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["weekly"]["next_to_next"]["date"], "expiry_type"] = "weekly"
        expiry["weekly"]["next_to_next"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["weekly"]["next_to_next"]["date"]}")

    ## Update the Monthly Expiry Dictionary 
    # Update the Monthly : Current Monthly
    if len(position_df.loc[position_df.expiry == expiry["monthly"]["current"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["monthly"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["monthly"]["current"]["date"], "expiry_type"] = "monthly"
        expiry["monthly"]["current"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["monthly"]["current"]["date"]}")

    # Update the Monthly : Next Monthly
    if len(position_df.loc[position_df.expiry == expiry["monthly"]["next"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["monthly"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["monthly"]["next"]["date"], "expiry_type"] = "monthly"
        expiry["monthly"]["next"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["monthly"]["next"]["date"]}")

    # Update the Monthly : Next_to_Next Monthly
    if len(position_df.loc[position_df.expiry == expiry["monthly"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["monthly"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["monthly"]["next_to_next"]["date"], "expiry_type"] = "monthly"
        expiry["monthly"]["next_to_next"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["monthly"]["next_to_next"]["date"]}")

    ## Update the Quarterly Expiry Dictionary 
    # Update the Quarterly : Current Quarterly
    if len(position_df.loc[position_df.expiry == expiry["quarterly"]["current"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["quarterly"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["quarterly"]["current"]["date"], "expiry_type"] = "quarterly"
        expiry["quarterly"]["current"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["quarterly"]["current"]["date"]}")

    # Update the Quarterly : Next Quarterly
    if len(position_df.loc[position_df.expiry == expiry["quarterly"]["next"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["quarterly"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["quarterly"]["next"]["date"], "expiry_type"] = "quarterly"
        expiry["quarterly"]["next"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["quarterly"]["next"]["date"]}")

    # Update the Quarterly : Next_to_Next Quarterly
    if len(position_df.loc[position_df.expiry == expiry["quarterly"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(
            f"Position Exist for the Expiry Date : {expiry["quarterly"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["quarterly"]["next_to_next"]["date"], "expiry_type"] = "quarterly"
        expiry["quarterly"]["next_to_next"]["position_exist"] = True

    else:
        print(f"No Position Exist for the Expiry Date : {expiry["quarterly"]["next_to_next"]["date"]}")

In [ ]:
await check_the_position_availablity()

In [ ]:
position_df

In [ ]:
current_daily_call, current_daily_put = await recommend_option_position(option_chain=current_daily)
next_daily_call, next_daily_put = await recommend_option_position(option_chain=next_daily)
next_to_next_daily_call, next_to_next_daily_put = await recommend_option_position(option_chain=next_to_next_daily)

current_weekly_call, current_weekly_put = await recommend_option_position(option_chain=current_weekly)
next_weekly_call, next_weekly_put = await recommend_option_position(option_chain=next_weekly)
next_to_next_weekly_call, next_to_next_weekly_put = await recommend_option_position(option_chain=next_to_next_weekly)

current_monthly_call, current_monthly_put = await recommend_option_position(option_chain=current_monthly)
next_monthly_call, next_monthly_put = await recommend_option_position(option_chain=next_monthly)
next_to_next_monthly_call, next_to_next_monthly_put = await recommend_option_position(option_chain=next_to_next_monthly)

current_quarterly_call, current_quarterly_put = await recommend_option_position(option_chain=current_quarterly)
next_quarterly_call, next_quarterly_put = await recommend_option_position(option_chain=next_quarterly)
next_to_next_quarterly_call, next_to_next_quarterly_put = await recommend_option_position(
    option_chain=next_to_next_quarterly)

In [ ]:
expiry.keys()

In [ ]:
if current_daily_call is not None and current_daily_put is not None and expiry["daily"]["current"][
    "position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_daily_call.iloc[0]["symbol"], quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_daily_put.iloc[0]["symbol"], quantity=1.00)
else:
    print(f"Cannot Place Orders for Daily Option as NO Matching Criteria Exist")

if next_daily_call is not None and next_daily_put is not None and expiry["daily"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_daily_call.iloc[0]["symbol"], quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_daily_put.iloc[0]["symbol"], quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Daily Option as NO Matching Criteria Exist")

if next_to_next_daily_call is not None and next_to_next_daily_put is not None and expiry["daily"]["next_to_next"][
    "position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_daily_call.iloc[0]["symbol"],
                                       quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_daily_put.iloc[0]["symbol"], quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Daily Option as NO Matching Criteria Exist")

############
if current_weekly_call is not None and current_weekly_put is not None and expiry["weekly"]["current"][
    "position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_weekly_call.iloc[0]["symbol"], quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_weekly_put.iloc[0]["symbol"], quantity=1.00)
else:
    print(f"Cannot Place Orders for Weekly Option as NO Matching Criteria Exist")

if next_weekly_call is not None and next_weekly_put is not None and expiry["weekly"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_weekly_call.iloc[0]["symbol"], quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_weekly_put.iloc[0]["symbol"], quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Weekly Option as NO Matching Criteria Exist")

if next_to_next_weekly_call is not None and next_to_next_weekly_put is not None and expiry["weekly"]["next_to_next"][
    "position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_weekly_call.iloc[0]["symbol"],
                                       quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_weekly_put.iloc[0]["symbol"],
                                       quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Weekly Option as NO Matching Criteria Exist")

############
if current_monthly_call is not None and current_monthly_put is not None and expiry["monthly"]["current"][
    "position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_monthly_call.iloc[0]["symbol"], quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_monthly_put.iloc[0]["symbol"], quantity=1.00)
else:
    print(f"Cannot Place Orders for Monthly Option as NO Matching Criteria Exist")

if next_monthly_call is not None and next_monthly_put is not None and expiry["monthly"]["next"][
    "position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_monthly_call.iloc[0]["symbol"], quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_monthly_put.iloc[0]["symbol"], quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Monthly Option as NO Matching Criteria Exist")

if next_to_next_monthly_call is not None and next_to_next_monthly_put is not None and expiry["monthly"]["next_to_next"][
    "position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_monthly_call.iloc[0]["symbol"],
                                       quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_monthly_put.iloc[0]["symbol"],
                                       quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Monthly Option as NO Matching Criteria Exist")

############
if current_quarterly_call is not None and current_quarterly_put is not None and expiry["quarterly"]["current"][
    "position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_quarterly_call.iloc[0]["symbol"], quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_quarterly_put.iloc[0]["symbol"], quantity=1.00)
else:
    print(f"Cannot Place Orders for Quarterly Option as NO Matching Criteria Exist")

if next_quarterly_call is not None and next_quarterly_put is not None and expiry["quarterly"]["next"][
    "position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_quarterly_call.iloc[0]["symbol"], quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_quarterly_put.iloc[0]["symbol"], quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Quarterly Option as NO Matching Criteria Exist")

if next_to_next_quarterly_call is not None and next_to_next_quarterly_put is not None and \
        expiry["quarterly"]["next_to_next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_quarterly_call.iloc[0]["symbol"],
                                       quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_quarterly_put.iloc[0]["symbol"],
                                       quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Quarterly Option as NO Matching Criteria Exist")


In [ ]:
position_df.info()

In [ ]:
position_df

In [ ]:
expiry_dataframe = position_df.drop(
    columns=["symbol", "avgPrice", "delta", "theta", "positionValue", "unrealisedPnl", "size", "markPrice",
             "createdTime", "seq", "updatedTime", "side", "curRealisedPnl", "positionStatus", "gamma",
             "vega", "option_type", "strike_price"]).copy()

In [ ]:
async def check_delta_computation():
    if round(sum(position_df.total_delta), 6) == round(sum(position_df.delta_hedging) / 2, 6):
        print(f"Total Delta is Computed Correctly !!! , Total Options Open Positional Detla is : ",
              f"{sum(position_df.total_delta)}",
              f"Total Delta Computed for Hedging : {sum(position_df.delta_hedging) / 2}")

    else:
        print(f"ERROR !!!! Delta Computed In-Correctly",
              f"Total Delta : {sum(position_df.total_delta)}",
              f"Total Delta Computed for Hedging : {sum(position_df.delta_hedging) / 2}")

In [ ]:
await check_delta_computation()

In [ ]:
expiry_dataframe

In [ ]:
await check_the_position_availablity()

In [ ]:
expiry

In [1]:
from ByBit import CoreLogic

In [2]:
btcCoreLogic = CoreLogic(
    api_url="https://api-demo.bybit.com",  # API URL
    api_endpoint="/v5/market/tickers",  # Endpoint URI
    api_keys="6TzoOeOuyIDiJN9cOu",  # API Keys
    api_secret="JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",  # API Secret
    baseCoin="BTC",  # BaseCoin neets be either BTC, ETH, SOL Only
    settleCoin="USDC",
    default_quantity=1.00,  # Default Short Quantity

    daily=True,  # Play on Daily Options: True or False
    weekly=True,  # Play on Weekly Options: True or False
    monthly=True,  # Play on Monthly Options: True or False
    quarterly=True,  # Play on Quarterly Options: True or False

    delta_value=0.07,  # Delta Value beyond which we would take our Positions
    min_bid_price=0.01,  # Minumum Bid Price to eliminate the Blank Bid Strikes
    initial_mark_price_diff=0.05,  # The Difference Between Mark & Bid Price 0.05 means 5%
    max_mark_price_diff=0.3,  # Maximum Difference between Mark & Bid Price
    mark_price_diff_steps=0.05,  # Mark & Ask Difference Percentage Incremental Step Size

    # Tolarated Delta Deavation Limit, after which we Negate the Delta Deavation
    # by hedging with Perpectual Futuers
    daily_delta_limit=0.10,  # Daily Delta Deviation Limit
    weekly_delta_limit=0.10,  # Weekly Delta Deviation Limit
    monthly_delta_limit=0.10,  # Monthly Delta Deviation Limit
    quarterly_delta_limit=0.10,  # Quarterly Delta Deviation Limit
)

In [3]:
await btcCoreLogic.core_logic_computation()

Class Variables Reinitialization Initiated
Tickers Data Retrieval Success : Data Count : 514
Data Fetching from ByBit API is Completed in : 737.2515960014425 ms 

Formatting & Processing of DataFrame is Completed in : 153.86801501153968 ms 

2024-10-28
2024-10-29
2024-10-30
2024-11-01
2024-11-08
2024-11-15
2024-11-29
2024-12-27
2025-03-28
2025-06-27
2025-09-26
Initial Current Month is  11
Original Length of Weekly Expiry List Array : 67, But Length of returned Array : 5 
Original Length of Monthly Expiry List Array : 20, But Length of returned Array : 4 
Original Length of Weekly Expiry List Array : 6, But Length of returned Array : 5 
Setting expiry[weekly][current][date] as : 2024-11-01 Setting expiry[weekly][current][exist] as : True
Setting expiry[weekly][next][date] as : 2024-11-08 Setting expiry[weekly][next][exist] as : True
Setting expiry[weekly][next_to_next][date] as : 2024-11-15 Setting expiry[weekly][next_to_next][exist] as : True
Setting expiry[monthly][current][date] as :

In [7]:
btcCoreLogic.position_df.sort_values("expiry", inplace=True, ignore_index=True)

In [8]:
btcCoreLogic.position_df.markPrice = round(btcCoreLogic.position_df.markPrice, 6)
btcCoreLogic.position_df.delta = round(btcCoreLogic.position_df.delta, 6)
btcCoreLogic.position_df.theta = round(btcCoreLogic.position_df.theta, 6)
btcCoreLogic.position_df.positionValue = round(btcCoreLogic.position_df.positionValue, 6)

btcCoreLogic.position_df.gamma = round(btcCoreLogic.position_df.gamma, 6)
btcCoreLogic.position_df.vega = round(btcCoreLogic.position_df.vega, 6)
btcCoreLogic.position_df.total_delta = round(btcCoreLogic.position_df.total_delta, 6)
btcCoreLogic.position_df.expiry_delta = round(btcCoreLogic.position_df.expiry_delta, 6)
btcCoreLogic.position_df.delta_hedging = round(btcCoreLogic.position_df.delta_hedging, 6)
btcCoreLogic.position_df.PerpFutureQty = round(btcCoreLogic.position_df.PerpFutureQty, 6)


In [9]:
btcCoreLogic.position_df

,symbol,avgPrice,delta,theta,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,...,vega,option_type,strike_price,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,BTC-28OCT24-65000-P,30.0,0.036074,62.084866,-22.672181,7.327819,22.672181,2024-10-25 12:35:02.855,671313656,2024-10-25 12:35:02.856,...,-2.310041,PUT,65000.0,2024-10-28,0.036074,0.1,daily,0.030788,False,0.030788
1,BTC-28OCT24-72000-C,25.0,-0.005286,12.209390,-2.678687,22.321313,2.678687,2024-10-25 12:35:02.479,671313655,2024-10-25 12:35:02.480,...,-0.443093,CALL,72000.0,2024-10-28,-0.005286,0.1,daily,0.030788,False,0.030788
2,BTC-29OCT24-64000-P,55.0,0.028493,39.899945,-22.722656,32.277344,22.722656,2024-10-27 04:14:30.357,671328974,2024-10-27 04:14:30.358,...,-2.993960,PUT,64000.0,2024-10-29,0.028493,0.1,daily,-0.042666,False,-0.042666
3,BTC-29OCT24-71000-C,20.0,-0.071158,86.382576,-66.413147,-46.413147,66.413147,2024-10-27 04:14:30.163,671328973,2024-10-27 04:14:30.163,...,-6.244760,CALL,71000.0,2024-10-29,-0.071158,0.1,daily,-0.042666,False,-0.042666
4,BTC-1NOV24-82000-C,125.0,-0.008830,13.631164,-15.087753,109.912247,15.087753,2024-10-18 06:00:37.171,671276188,2024-10-18 06:00:37.172,...,-1.835447,CALL,82000.0,2024-11-01,-0.008830,0.1,weekly,-0.000496,False,-0.000496
5,BTC-1NOV24-58000-P,195.0,0.008334,10.933970,-12.735945,182.264055,12.735945,2024-10-18 06:00:37.377,671276189,2024-10-18 06:00:37.378,...,-1.744602,PUT,58000.0,2024-11-01,0.008334,0.1,weekly,-0.000496,False,-0.000496
6,BTC-8NOV24-88000-C,215.0,-0.033605,29.604557,-117.293687,97.706313,117.293687,2024-10-18 06:00:37.732,671276190,2024-10-18 06:00:37.732,...,-9.083390,CALL,88000.0,2024-11-08,-0.033605,0.1,weekly,-0.003052,False,-0.003052
7,BTC-8NOV24-54000-P,290.0,0.030553,25.557395,-106.510640,183.489360,106.510640,2024-10-18 06:00:38.007,671276191,2024-10-18 06:00:38.007,...,-8.397229,PUT,54000.0,2024-11-08,0.030553,0.1,weekly,-0.003052,False,-0.003052
8,BTC-15NOV24-84000-C,290.0,-0.089196,43.478963,-391.838163,-101.838163,391.838163,2024-10-25 23:50:02.937,671321519,2024-10-25 23:50:02.937,...,-24.847816,CALL,84000.0,2024-11-15,-0.089196,0.1,weekly,-0.043053,False,-0.043053
9,BTC-15NOV24-54000-P,190.0,0.046143,25.377443,-197.034239,-7.034239,197.034239,2024-10-25 23:50:03.277,671321520,2024-10-25 23:50:03.277,...,-14.898025,PUT,54000.0,2024-11-15,0.046143,0.1,weekly,-0.043053,False,-0.043053


In [10]:
expiry_df = btcCoreLogic.position_df.drop(
    columns=["symbol", "avgPrice", "delta", "theta", "positionValue", "unrealisedPnl", "size", "markPrice",
             "createdTime", "seq", "updatedTime", "side", "curRealisedPnl", "positionStatus", "gamma",
             "vega", "option_type", "strike_price"]).copy()

In [11]:
expiry_df.drop_duplicates("expiry", inplace=True, ignore_index=True)

In [12]:
expiry_df

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,2024-10-28,0.036074,0.1,daily,0.030788,False,0.030788
1,2024-10-29,0.028493,0.1,daily,-0.042666,False,-0.042666
2,2024-11-01,-0.008830,0.1,weekly,-0.000496,False,-0.000496
3,2024-11-08,-0.033605,0.1,weekly,-0.003052,False,-0.003052
4,2024-11-15,-0.089196,0.1,weekly,-0.043053,False,-0.043053
5,2024-11-29,0.051796,0.1,monthly,0.005261,False,0.005261
6,2024-12-27,-0.026160,0.1,monthly,-0.008680,False,-0.008680
7,2025-03-28,-0.018889,0.1,quarterly,-0.006896,False,-0.006896
8,2025-09-26,-0.023272,0.1,quarterly,-0.018010,False,-0.018010


In [13]:
btcCoreLogic.position_df.loc[btcCoreLogic.position_df.expiry == expiry_df.loc[0, "expiry"]]

,symbol,avgPrice,delta,theta,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,...,vega,option_type,strike_price,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,BTC-28OCT24-65000-P,30.0,0.036074,62.084866,-22.672181,7.327819,22.672181,2024-10-25 12:35:02.855,671313656,2024-10-25 12:35:02.856,...,-2.310041,PUT,65000.0,2024-10-28,0.036074,0.1,daily,0.030788,False,0.030788
1,BTC-28OCT24-72000-C,25.0,-0.005286,12.209390,-2.678687,22.321313,2.678687,2024-10-25 12:35:02.479,671313655,2024-10-25 12:35:02.480,...,-0.443093,CALL,72000.0,2024-10-28,-0.005286,0.1,daily,0.030788,False,0.030788


In [14]:
import numpy as np

avgPrice = np.array([-5000.00 for _ in range(len(expiry_df))])
markPrice = np.array([-5000.00 for _ in range(len(expiry_df))])
unrealisedPnl = np.array([-5000.00 for _ in range(len(expiry_df))])

expiry_df["avgPrice"] = avgPrice
expiry_df["markPrice"] = markPrice
expiry_df["unrealisedPnl"] = unrealisedPnl


In [15]:
for i in range(len(expiry_df)):
    expiry_df.loc[i, "markPrice"] = sum(btcCoreLogic.position_df.loc[
                                            btcCoreLogic.position_df.expiry == expiry_df.loc[i, "expiry"], "markPrice"])
    expiry_df.loc[i, "avgPrice"] = sum(btcCoreLogic.position_df.loc[
                                           btcCoreLogic.position_df.expiry == expiry_df.loc[i, "expiry"], "avgPrice"])
    expiry_df.loc[i, "unrealisedPnl"] = sum(btcCoreLogic.position_df.loc[
                                                btcCoreLogic.position_df.expiry == expiry_df.loc[
                                                    i, "expiry"], "unrealisedPnl"])


In [16]:
expiry_df

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty,avgPrice,markPrice,unrealisedPnl
0,2024-10-28,0.036074,0.1,daily,0.030788,False,0.030788,55.0,25.350868,29.649132
1,2024-10-29,0.028493,0.1,daily,-0.042666,False,-0.042666,75.0,89.135803,-14.135803
2,2024-11-01,-0.008830,0.1,weekly,-0.000496,False,-0.000496,320.0,27.823698,292.176302
3,2024-11-08,-0.033605,0.1,weekly,-0.003052,False,-0.003052,505.0,223.804327,281.195673
4,2024-11-15,-0.089196,0.1,weekly,-0.043053,False,-0.043053,480.0,588.872402,-108.872402
5,2024-11-29,0.051796,0.1,monthly,0.005261,False,0.005261,760.0,530.086709,229.913291
6,2024-12-27,-0.026160,0.1,monthly,-0.008680,False,-0.008680,410.0,71.758658,676.482685
7,2025-03-28,-0.018889,0.1,quarterly,-0.006896,False,-0.006896,465.0,330.822596,134.177404
8,2025-09-26,-0.023272,0.1,quarterly,-0.018010,False,-0.018010,155.0,410.444516,-255.444516


In [17]:
delta_risk_multiplier: float = 2.0
total_perpFutures: float = 0.0
temp_count: float = 0.0
total_delta_risk_magnitude: float = 0.0

total_perpFutures = round(sum(expiry_df.loc[expiry_df.hedging_required == True, "PerpFutureQty"]), 6)
total_perpFutures

0

In [18]:
daily_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "daily"].PerpFutureQty), 6)
daily_delta_risk_magnitude = delta_risk_multiplier * 0.1

if daily_delta_risk > daily_delta_risk_magnitude:

    total_delta_risk_magnitude = total_delta_risk_magnitude + daily_delta_risk

    print(f"Attention !!!! : Daily Delta Risk Paramater Exceeded : {daily_delta_risk_magnitude}, "
          f"Risk Magnitude : {daily_delta_risk}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Daily Delta Risk Paramater is within Limit : {daily_delta_risk_magnitude} ",
          f"Risk Magnitude : {daily_delta_risk}, Total Risk Magnitude: {total_delta_risk_magnitude}")

Daily Delta Risk Paramater is within Limit : 0.2  Risk Magnitude : -0.011878, Total Risk Magnitude: 0.0


In [19]:
expiry_df[expiry_df.expiry_type == "weekly"]

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty,avgPrice,markPrice,unrealisedPnl
2,2024-11-01,-0.008830,0.1,weekly,-0.000496,False,-0.000496,320.0,27.823698,292.176302
3,2024-11-08,-0.033605,0.1,weekly,-0.003052,False,-0.003052,505.0,223.804327,281.195673
4,2024-11-15,-0.089196,0.1,weekly,-0.043053,False,-0.043053,480.0,588.872402,-108.872402


In [20]:
weekly_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "weekly"].PerpFutureQty), 6)
weekly_delta_risk_magnitude = delta_risk_multiplier * 0.1

if weekly_delta_risk > weekly_delta_risk_magnitude:

    total_delta_risk_magnitude = total_delta_risk_magnitude + weekly_delta_risk

    print(f"Attention !!!! : Weekly Delta Risk Paramater Exceeded : {weekly_delta_risk}, "
          f"Risk Magnitude : {weekly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Weekly Delta Risk Paramater is within Limit : {weekly_delta_risk} ",
          f"Risk Magnitude : {weekly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

Weekly Delta Risk Paramater is within Limit : -0.046601  Risk Magnitude : 0.2, Total Risk Magnitude: 0.0


In [21]:
expiry_df[expiry_df.expiry_type == "monthly"]

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty,avgPrice,markPrice,unrealisedPnl
5,2024-11-29,0.051796,0.1,monthly,0.005261,False,0.005261,760.0,530.086709,229.913291
6,2024-12-27,-0.026160,0.1,monthly,-0.008680,False,-0.008680,410.0,71.758658,676.482685


In [22]:
monthly_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "monthly"].PerpFutureQty), 6)
monthly_delta_risk_magnitude = delta_risk_multiplier * 0.1

if monthly_delta_risk > monthly_delta_risk_magnitude:
    total_delta_risk_magnitude = total_delta_risk_magnitude + monthly_delta_risk

    print(f"Attention !!!! : Monthly Delta Risk Paramater Exceeded : {monthly_delta_risk}, "
          f"Risk Magnitude : {monthly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Monthly Delta Risk Paramater is within Limit : {monthly_delta_risk} ",
          f"Risk Magnitude : {monthly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

Monthly Delta Risk Paramater is within Limit : -0.003419  Risk Magnitude : 0.2, Total Risk Magnitude: 0.0


In [23]:
expiry_df[expiry_df.expiry_type == "quarterly"]

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty,avgPrice,markPrice,unrealisedPnl
7,2025-03-28,-0.018889,0.1,quarterly,-0.006896,False,-0.006896,465.0,330.822596,134.177404
8,2025-09-26,-0.023272,0.1,quarterly,-0.018010,False,-0.018010,155.0,410.444516,-255.444516


In [24]:
quarterly_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "quarterly"].PerpFutureQty), 6)
quarterly_delta_risk_magnitude = delta_risk_multiplier * 0.1

if quarterly_delta_risk > quarterly_delta_risk_magnitude:

    total_delta_risk_magnitude = total_delta_risk_magnitude + quarterly_delta_risk

    print(f"Attention !!!! : Quarterly Delta Risk Paramater Exceeded : {quarterly_delta_risk}, "
          f"Risk Magnitude : {quarterly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Quarterly Delta Risk Paramater is within Limit : {quarterly_delta_risk} ",
          f"Risk Magnitude : {quarterly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

Quarterly Delta Risk Paramater is within Limit : -0.024906  Risk Magnitude : 0.2, Total Risk Magnitude: 0.0


In [25]:
sum((btcCoreLogic.position_df["size"] * btcCoreLogic.position_df["avgPrice"]) - (
            btcCoreLogic.position_df["markPrice"] * btcCoreLogic.position_df["size"]))


1265.141765

In [26]:
sum((btcCoreLogic.position_df["size"] * btcCoreLogic.position_df["avgPrice"]) - (
            btcCoreLogic.position_df["markPrice"] * btcCoreLogic.position_df["size"]))


1265.141765

In [27]:
((
             total_perpFutures >= total_delta_risk_magnitude) and total_perpFutures != 0.0), total_perpFutures >= total_delta_risk_magnitude


(False, True)

In [28]:
total_delta_risk_magnitude, total_perpFutures,


(0.0, 0)

In [29]:
from ByBit import ByBitAPI

baseCoin = "BTC"
ByBitAPI = ByBitAPI(default_quantity=1.0,
                    api_url="https://api-demo.bybit.com",
                    api_key="6TzoOeOuyIDiJN9cOu",
                    api_secret="JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",
                    baseCoin="BTC",
                    settleCoin="USDC")

In [30]:
try:
    PerpFutures_json_data = await ByBitAPI.get_PerpFutures_Position()
    if len(PerpFutures_json_data.json()["result"]["list"]) > 0:
        PerpFutures_df = await ByBitAPI.format_perpfutures()

        if PerpFutures_df.loc[0, "side"] == "BUY":
            PerpFutures_position = float(PerpFutures_df.loc[0, "size"])
        elif PerpFutures_df.loc[0, "side"] == "SELL":
            PerpFutures_position = float(PerpFutures_df.loc[0, "size"]) * (-1.0)

        print(f"Existing Perpetual Futures Position Size : {PerpFutures_position}")

    else:
        PerpFutures_position = 0.0
        print(
            f"No {baseCoin}, Perpectual Futires position Exists, Setting Value for 'PerpFutures_position'  :  {PerpFutures_position}")
except Exception as e:
    print(f"Error in Fetcting the Perpectual Futures Position Error Details : {e}")

Parameter String is : 17300440398176TzoOeOuyIDiJN9cOu5000category=linear&baseCoin=BTC&settleCoin=USDC

Generated Signature is : 31062512cccdcb0718f86a14e41a2e5a3c71267340c57478d516f7bacf92c220

Perpectual Futures Position Info API Signature : 31062512cccdcb0718f86a14e41a2e5a3c71267340c57478d516f7bacf92c220

API Header is : {'X-BAPI-API-KEY': '6TzoOeOuyIDiJN9cOu', 'X-BAPI-SIGN': '31062512cccdcb0718f86a14e41a2e5a3c71267340c57478d516f7bacf92c220', 'X-BAPI-SIGN-TYPE': '2', 'X-BAPI-TIMESTAMP': '1730044039817', 'X-BAPI-RECV-WINDOW': '5000', 'Content-Type': 'application/json'} 

Final URL is : https://api-demo.bybit.com/v5/position/list?category=linear&baseCoin=BTC&settleCoin=USDC
Positions Info Length is : 0
No BTC, Perpectual Futires position Exists, Setting Value for 'PerpFutures_position'  :  0.0


In [31]:
PerpFutures_position, total_perpFutures, total_delta_risk_magnitude

(0.0, 0, 0.0)

In [32]:
if total_perpFutures >= total_delta_risk_magnitude:
    # Compute the Perpectual Futures Position Requirements
    PerpFut_Requirement = total_perpFutures
elif total_perpFutures < total_delta_risk_magnitude:
    PerpFut_Requirement = total_delta_risk_magnitude


In [ ]:
PerpFutures_position, total_perpFutures,

In [41]:
PerpFut_Requirement = -0.1

In [42]:
if PerpFut_Requirement == 0:
    print(f"No Perpectual Futures Position is Required as Requirements is : {PerpFut_Requirement}")
elif PerpFut_Requirement > 0:
    PerpFut_Requirement = round((PerpFut_Requirement * (-1.0)), 3)
    print(f"We need to Go Short PerpFuture with following Quantity : {abs(PerpFut_Requirement)}, "
          f"Setting PerpFuture Quantity as : {PerpFut_Requirement} ")
elif PerpFut_Requirement < 0:
    PerpFut_Requirement = round((PerpFut_Requirement * (-1.0)), 3)
    print(f"We need to Go Long PerpFuture with following Quantity : {abs(PerpFut_Requirement)}, "
          f"Setting PerpFuture Quantity as : {PerpFut_Requirement} ")

We need to Go Long PerpFuture with following Quantity : 0.1, Setting PerpFuture Quantity as : 0.1 


In [44]:
PerpFut_Position = 0.0
adjustment_quantity = 0.0

In [107]:
def check_PerpFutures_Quantity(PerpFut_Position, PerpFut_Requirement):
    adjustment_quantity = 0.0
    if PerpFut_Position == 0.0:

        print(
            f"No PerpFuture Position Exist, PerpFuture Position {PerpFut_Position}, Will Process the PerpFuture Requirements as is")

        if PerpFut_Requirement == 0:
            print(f"No PerpFuture Position is Required : {PerpFut_Requirement}, No Order Processing is Required")

        elif PerpFut_Requirement > 0:
            print(f"Placing a 'BUY' Order for PerpFuture with following Quantity : {PerpFut_Requirement}")

        elif PerpFut_Requirement < 0:
            print(f"Placing a 'SELL' Order for PerpFuture with following Quantity : {PerpFut_Requirement}")

    elif PerpFut_Position > 0.0:

        if PerpFut_Requirement == 0:
            adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)

            print(
                f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                f"So Futures Adjustment 'SELL' Quantity is : {adjustment_quantity}")

        elif PerpFut_Requirement > 0:

            if PerpFut_Position > PerpFut_Requirement:
                adjustment_quantity = -round((PerpFut_Position - PerpFut_Requirement), 3)
            elif PerpFut_Position < PerpFut_Requirement:
                adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)

            if adjustment_quantity < 0:
                print(
                    f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                    f"So Futures Adjustment 'SELL' Quantity is : {adjustment_quantity}")
            else:
                print(
                    f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                    f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")

        elif PerpFut_Requirement < 0:
            adjustment_quantity = round((-PerpFut_Position + PerpFut_Requirement), 3)

            if adjustment_quantity < 0:
                print(
                    f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                    f"So Futures Adjustment 'SELL' Quantity is : {adjustment_quantity}")
            else:
                print(
                    f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                    f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")


    elif PerpFut_Position < 0.0:

        if PerpFut_Requirement == 0:
            adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)

            print(
                f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")

        elif PerpFut_Requirement > 0:
            adjustment_quantity = round((-PerpFut_Position + PerpFut_Requirement), 3)

            assert adjustment_quantity > 0.0

            print(
                f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")


        elif PerpFut_Requirement < 0:
            if PerpFut_Position > PerpFut_Requirement:
                adjustment_quantity = -round((PerpFut_Position - PerpFut_Requirement), 3)
            elif PerpFut_Position < PerpFut_Requirement:
                adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)

            if adjustment_quantity < 0:
                print(
                    f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                    f"So Futures Adjustment 'SELL' Quantity is : {adjustment_quantity}")
            else:
                print(
                    f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                    f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")

    return adjustment_quantity


In [108]:
PerpFut_Position = 0.02
PerpFut_Requirement = 0.01
if PerpFut_Position > PerpFut_Requirement:
    adjustment_quantity = -round((PerpFut_Position - PerpFut_Requirement), 3)
elif PerpFut_Position < PerpFut_Requirement:
    adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)

adjustment_quantity

-0.01

In [109]:
PerpFut_Position = 0.12
PerpFut_Requirement = -0.1

PerpFut_Position, PerpFut_Requirement, round((-PerpFut_Position + PerpFut_Requirement), 3)

(0.12, -0.1, -0.22)

In [110]:
check_PerpFutures_Quantity(0.0, 0.0)

No PerpFuture Position Exist, PerpFuture Position 0.0, Will Process the PerpFuture Requirements as is
No PerpFuture Position is Required : 0.0, No Order Processing is Required


0.0

In [111]:
check_PerpFutures_Quantity(0.1, 0.2)

Since Existing PerpFuture Position Exist : 0.1, and PerpFuture Requirements : 0.2,So Futures Adjustment 'BUY' Quantity is : 0.1


0.1

In [112]:
check_PerpFutures_Quantity(0.2, 0.1)

Since Existing PerpFuture Position Exist : 0.2, and PerpFuture Requirements : 0.1,So Futures Adjustment 'SELL' Quantity is : -0.1


-0.1

In [113]:
check_PerpFutures_Quantity(-0.1, -0.2)

Since Existing PerpFuture Position Exist : -0.1, and PerpFuture Requirements : -0.2,So Futures Adjustment 'SELL' Quantity is : -0.1


-0.1

In [114]:
check_PerpFutures_Quantity(-0.2, -0.1)

Since Existing PerpFuture Position Exist : -0.2, and PerpFuture Requirements : -0.1,So Futures Adjustment 'BUY' Quantity is : 0.1


0.1

In [115]:
check_PerpFutures_Quantity(-0.1, 0.2)

Since Existing PerpFuture Position Exist : -0.1, and PerpFuture Requirements : 0.2,So Futures Adjustment 'BUY' Quantity is : 0.3


0.3

In [116]:
check_PerpFutures_Quantity(0.1, -0.2)

Since Existing PerpFuture Position Exist : 0.1, and PerpFuture Requirements : -0.2,So Futures Adjustment 'SELL' Quantity is : -0.3


-0.3

In [117]:
check_PerpFutures_Quantity(-0.1, -0.1)

Since Existing PerpFuture Position Exist : -0.1, and PerpFuture Requirements : -0.1,So Futures Adjustment 'BUY' Quantity is : 0.0


0.0

In [118]:
check_PerpFutures_Quantity(0.1, 0.1)

Since Existing PerpFuture Position Exist : 0.1, and PerpFuture Requirements : 0.1,So Futures Adjustment 'BUY' Quantity is : 0.0


0.0

In [119]:
check_PerpFutures_Quantity(-0.2, 0.1)

Since Existing PerpFuture Position Exist : -0.2, and PerpFuture Requirements : 0.1,So Futures Adjustment 'BUY' Quantity is : 0.3


0.3

In [ ]:
sum(expiry_df.loc[expiry_df.hedging_required == True, "PerpFutureQty"])

In [ ]:
time.sleep(10)

In [ ]:
await asyncio.sleep(5)

In [ ]:
import datetime
import time
import asyncio

In [ ]:
utcnow = datetime.datetime.now(datetime.UTC).
utcnow

In [ ]:
if utcnow.minute % 5 == 0 and utcnow.second == 0:
    print(executing
    Function
    Current
    time
    {str(utcnow)})

    else:
    await asyncio.sleep()

In [ ]:
str(utcnow), (5 - (datetime.datetime.now(datetime.UTC).minute % 5)) * 60, datetime.datetime.now(datetime.UTC).second


In [ ]:
(5 - (datetime.datetime.now(datetime.UTC).minute % 5)) * 60 + datetime.datetime.now(datetime.UTC).second


In [ ]:
i = 0
while i <= 3:
    current_utc_time = datetime.datetime.now(datetime.UTC)
    if (current_utc_time.minute % 5) == 0:
        print(f"Executimg Function {str(current_utc_time)},"
              f"Wait Timer : {(4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second)}")
        await asyncio.sleep(60)
    else:
        print(f"Entering Sleep at {str(current_utc_time)}",
              f"Wait Timer : {(4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second)}")
        await asyncio.sleep((4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second))
    i = i + 1

In [ ]:
current_utc_time = datetime.datetime.now(datetime.UTC)

(4 - (current_utc_time.minute % 5)) * 60 + (
            60 - current_utc_time.second), current_utc_time.minute, current_utc_time.second

In [ ]:
current_utc_time

In [ ]:
import datetime

datetime.datetime.now(datetime.UTC)